In [1]:
#instantiate cursor and load libraries
import json
import pandas as pd
import requests
import mysql.connector
import time
import config
import requests
from pandas.io.json import json_normalize

In [2]:
import numpy as np

In [3]:
csvs = ['LA_water_data.csv', '']

In [28]:
def read_csv(csvs):
    for csv in csvs:
        df = pd.read_csv(csv)
        df.rename(columns={'ActivityLocation/LatitudeMeasure': 'ActivityLatitude',
                           'ActivityLocation/LongitudeMeasure': 'ActivityLongitude',
                           'CharacteristicName':'Contaminant',
                           'ResultMeasureValue':'DetectedLevel',
                           'ResultMeasure/MeasureUnitCode':'LevelUnit'}, inplace=True)
        rows = ['Lead', 'Copper']
        copper_and_lead_df = df[df['Contaminant'].isin(rows)]
        final_copper_and_lead_df = pd.DataFrame(copper_and_lead_df[['ActivityIdentifier',
                                                                    'ActivityLatitude',
                                                                    'ActivityLongitude',
                                                                    'AnalysisStartDate',
                                                                    'Contaminant',
                                                                    'DetectedLevel',
                                                                    'LevelUnit',
                                                                    'ActivityMediaSubdivisionName']])
        
        final_copper_and_lead_df = final_copper_and_lead_df[pd.notnull(final_copper_and_lead_df['DetectedLevel'])]
        final_copper_and_lead_df['DetectedLevel'] = final_copper_and_lead_df.DetectedLevel.astype(float)
        final_copper_and_lead_df = final_copper_and_lead_df[pd.notnull(final_copper_and_lead_df['DetectedLevel'])]
        final_copper_and_lead_df = final_copper_and_lead_df.where((pd.notnull(final_copper_and_lead_df)), None)
        final_copper_and_lead_df = final_copper_and_lead_df.replace('ND', 0)
        
        print('all csvs read')
        return final_copper_and_lead_df

# read_csv(csvs)

In [4]:
#set up mysql connection
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password)
    
cursor = cnx.cursor()

In [19]:
#create database
from mysql.connector import errorcode
db_name = 'water'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [5]:
#specify database for connection
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = "water")

cursor = cnx.cursor()

#create table
DB_NAME = 'water'

create_query = """
      CREATE TABLE water_table
    ( ActivityIdentifier VARCHAR(50),
      ActivityLatitude FLOAT(10),
      ActivityLongitude FLOAT(10),
      AnalysisStartDate DATE,
      Contaminant VARCHAR(20),
      DetectedLevel FLOAT(10),
      LevelUnit VARCHAR(10),
      ActivityMediaSubdivisionName VARCHAR(30)
      )
    """


In [6]:
cursor.execute(create_query)

In [26]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = "water")

cursor = cnx.cursor()

def insert_data(dataframe):
    tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO water_table (ActivityIdentifier, 
                                                ActivityLatitude, 
                                                ActivityLongitude, 
                                                AnalysisStartDate,
                                                Contaminant,
                                                DetectedLevel,
                                                LevelUnit,
                                                ActivityMediaSubdivisionName
                                                ) 
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"""
    cursor.executemany(insert_statement, tuples)
    cnx.commit()

In [27]:
insert_data(read_csv(csvs))

/Users/cblessen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (5,7,8,9,10,11,12,14,15,17,19,20,21,23,24,25,26,31,32,33,54,57,58,60,61,62,63,66,67,68,69,70,72,73,75,76,78,80,82,83,84,85,88,91,94,95,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,148) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


all csvs read
